<a href="https://colab.research.google.com/github/henriquecm02/Projeto_Imers-o_AluraGoogle_SI.O.S/blob/main/codigocolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando e Importando Bibliotecas Úteis ao Projeto.

In [ ]:
!pip install streamlit
!pip install genai
import os
import streamlit as st
import google.generativeai as genai
import base64
import imaplib
import smtplib
from email.mime.text import MIMEText
import threading
import time


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.5/816.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.24.0 which is incompatible.


## Conectando ao Google AI Studio, Defindo o Modelo de LLM e Inicializando os Protocolos de Segurança e Login.

> Adicionar aspas

> Adicionar aspas





In [ ]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK',
    'DANGEROUS' : 'BLOCK_NONE'
    }

### INICIALMENTE SERÃO ADICIONADOS FILTROS POR SE TRATAR DE UMA FERRAMENTA DE SEGURANÇA E ESTAR EM FASE DE PROTOTIPAGEM E DESENVOLVIMENTO.
 **Por ser permitido o acesso a menores de 18 anos

In [ ]:
generation_config = {
  "candidate_count": 1,
  "temperature": 1,
}

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Carregar a chave API de variável de ambiente
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    st.error("A variável de ambiente 'GOOGLE_API_KEY' não está definida. Configure-a no Google Colab.")
genai.configure(api_key=GOOGLE_API_KEY)

# Credenciais para API do Google
# **Substitua pelo caminho do seu arquivo JSON no Google Colab!**
# Você pode usar a funcionalidade de upload de arquivos do Colab para fazer upload do seu arquivo credentials.json.
SERVICE_ACCOUNT_FILE = '/content/credentials.json' # Substitua pelo caminho correto
SCOPES = ['https://www.googleapis.com/auth/admin.reports.audit.readonly', 'https://www.googleapis.com/auth/admin.directory.user.readonly']
# Modelo de IA a ser utilizado
MODEL_NAME = "gemini-1.0-pro"



## Preenchendo Dicionários que Serão Utilizados Ao Longo da Execução do Código.

In [ ]:
# Definição das personas e estilos de comunicação para o chatbot
system_instructions = {
    1: {
        "persona": "Professor de Segurança da Informação",
        "tone": "informative",
        "style": "friendly"
    },
    2: {
        "persona": "Engenheiro de Software",
        "tone": "technical",
        "style": "professional"
    },
    3: {
        "persona": "Especialista em Segurança da Informação",
        "tone": "authoritative",
        "style": "formal"
    }
}

'''# Carregar conteúdos de arquivos externos
# Adaptado para Colab - carregando os arquivos de texto a partir do seu Google Drive
from google.colab import drive
drive.mount('/content/drive')

def carregar_conteudo(arquivo):
    with open(f'/content/drive/My Drive/{arquivo}', "r", encoding='utf-8') as f: # Substitua 'caminho/para/seu/arquivo' pelo caminho real no seu Drive
        return f.read()'''

# Conteúdos pré-definidos para a persona de Professor de Segurança da Informação
conteudos = {
    1: "",
    2: "",
    3: "",
    4: "",
    5: "",
}

# Métodos de integração para o  perfíl  3
metodos = {
    1: "",
    2: "",
    3: "",
}


##   Configurando a Segurança das Contas Google Através do Google AI Studio, Pedindo Acesso a Conta de Email e realizando o Login Caso o Usúario Opte.

In [ ]:
# Configurações de email (substituir com suas informações)
EMAIL_ADDRESS = os.environ.get("EMAIL_ADDRESS")
EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD")
SMTP_SERVER = os.environ.get("SMTP_SERVER")
IMAP_SERVER = os.environ.get("IMAP_SERVER")

# Verificar se as variáveis de ambiente de email estão definidas
if not all([EMAIL_ADDRESS, EMAIL_PASSWORD, SMTP_SERVER, IMAP_SERVER]):
    st.error("Variáveis de ambiente de email não estão configuradas corretamente. Configure-as no Google Colab.")

# Função para conectar ao servidor IMAP e recuperar emails
def get_emails(criteria):
    try:
        with imaplib.IMAP4_SSL(IMAP_SERVER) as imap:
            imap.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            status, data = imap.select("INBOX")
            if status == 'OK':
                imap.search(None, criteria)
                status, data = imap.fetch(data[0][0], '(RFC822)')
                for msg_id, msg_data in data:
                    for response_part in msg_data:
                        if isinstance(response_part, bytes):
                            message = email.message_from_string(response_part.decode('utf-8'))
                            yield message
    except Exception as e:
        st.error(f"Erro ao conectar ao servidor de email: {e}")

# Função para enviar email de alerta
def send_alert_email(subject, body):
    try:
        message = MIMEText(body)
        message['Subject'] = subject
        message['From'] = EMAIL_ADDRESS
        message['To'] = EMAIL_ADDRESS
        with smtplib.SMTP(SMTP_SERVER) as server:
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.sendmail(EMAIL_ADDRESS, EMAIL_ADDRESS, message.as_string())
    except Exception as e:
        st.error(f"Erro ao enviar email de alerta: {e}")

# Função para conectar à API do Google
def connect_to_google_api():
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return build('admin', 'reports_v1', credentials=credentials), build('admin', 'directory_v1', credentials=credentials)

# Função para monitorar atividades de login
def monitor_login_activities(reports_service, start_time):
    results = reports_service.activities().list(userKey='all', applicationName='login', startTime=start_time).execute()
    activities = results.get('items', [])
    for activity in activities:
        event_name = activity['events'][0]['name']
        if event_name == 'login_failure':
            st.warning(f"Falha de login detectada: {activity}")
            send_alert_email("Alerta de Segurança: Falha de Login", f"Detalhes da falha: {activity}")

# Função para monitorar usuários
def monitor_users(directory_service):
    results = directory_service.users().list(customer='my_customer', orderBy='email').execute()
    users = results.get('users', [])
    for user in users:
        st.write(f"Usuário: {user['name']['fullName']} ({user['primaryEmail']})")
        # Implementar verificações adicionais de segurança aqui
        # ...

# Interface de usuário no Streamlit
st.title("S1.0.S - Seu Assistente Inteligente de Segurança da Informação")

# Seção de boas-vindas e seleção de perfil de usuário
reconhecimento = st.selectbox("Qual o seu perfil de usuário?", list(system_instructions.keys()))

# Exibição de conteúdo com base no perfil
if reconhecimento == 1:
    st.header("Conteúdo Básico de Segurança da Informação")
    conteudo_selecionado = st.selectbox("Escolha um tópico:", conteudos.keys())
    if conteudo_selecionado:
        st.write(conteudos[conteudo_selecionado])
elif reconhecimento in [2, 3]:
    st.header("Recursos para Profissionais de Segurança e Desenvolvimento")
    metodo_selecionado = st.selectbox("Escolha um tópico:", metodos.keys())
    if metodo_selecionado:
        st.write(metodos[metodo_selecionado])

# Seção de monitoramento de email
if st.checkbox("Ativar monitoramento de email?"):
    # Opções de critérios de pesquisa de email
    col1, col2 = st.columns(2)
    from_address = col1.text_input("De:", placeholder="remetente@exemplo.com (opcional)")
    to_address = col2.text_input("Para:", placeholder="destinatario@exemplo.com (opcional)")
    subject_contains = st.text_input("Assunto contém:", placeholder="palavras-chave (opcional)")
    body_contains = st.text_input("Corpo contém:", placeholder="palavras-chave (opcional)")
    # Botão para iniciar o monitoramento
    if st.button("Iniciar monitoramento"):
        def monitor_emails():
            criteria = f'(OR (FROM "{from_address}") (TO "{to_address}") (SUBJECT "{subject_contains}") (BODY "{body_contains}"))'
            for message in get_emails(criteria):
                st.write(f"Novo email recebido: {message['Subject']}")
                # Implementar lógica de detecção de phishing/malware aqui
                # ...
                # Enviar alerta se necessário
                # send_alert_email("Alerta de segurança!", "Email suspeito recebido!")
        thread = threading.Thread(target=monitor_emails)
        thread.start()

# Seção de monitoramento de contas Google
st.header("Monitoramento de Contas Google")
if st.checkbox("Ativar monitoramento de contas Google?"):
    reports_service, directory_service = connect_to_google_api()
    start_time = (time.time() - 3600) * 1000  # Última hora em milissegundos
    def monitor_google_accounts():
        while True:
            monitor_login_activities(reports_service, start_time)
            monitor_users(directory_service)
            time.sleep(60)  # Verificar a cada minuto
    thread = threading.Thread(target=monitor_google_accounts)
    thread.start()


In [ ]:
# Seção de monitoramento de contas Google
st.header("Monitoramento de Contas Google")
if st.checkbox("Ativar monitoramento de contas Google?"):
    reports_service, directory_service = connect_to_google_api()
    start_time = (time.time() - 3600) * 1000  # Última hora em milissegundos
    def monitor_google_accounts():
        while True:
            monitor_login_activities(reports_service, start_time)
            monitor_users(directory_service)
            time.sleep(60)  # Verificar a cada minuto
    thread = threading.Thread(target=monitor_google_accounts)
    thread.start()

# Chatbot com tom e estilo adequados
st.header("Chatbot de Segurança da Informação")
system_instruction = system_instructions[reconhecimento]
chat_session = genai.ChatSession(MODEL_NAME)
while True:
    prompt = st.text_input("Como posso auxiliá-lo(a) hoje?", key="prompt")
    if prompt == "fim":
        break
    response = chat_session.send_message(prompt)
    st.write(f"Resposta: {response.text}")

### Configuração do Chatbot e Definição dos varios tipos de Perfis.